# Parâmetros da API

Há alguns parâmetros que podem e devem ser configurados para refinar os prompts enviados à API de complementação (completions). Abaixo está a célula de configuração de ambiente como demostrado em [Configuração de Ambiente]('environment_setup.ipynb').

In [2]:
import openai
import configparser

config = configparser.ConfigParser()
config.read('config.ini') # caminho do arquivo de config .ini

openai.api_key = config['AZURE_CONFIG']['AZURE_OPENAI_KEY']
openai.api_base = config["AZURE_CONFIG"]["AZURE_OPENAI_ENDPOINT"]
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # pode mudar futuramente
DEPLOYMENT_NAME = config["AZURE_CONFIG"]["DEPLOYMENT_NAME"]

## Corpo do Request

Abaixo, segue [tabela disponibilizada pela Microsoft](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference) com um apanhado geral sobre os campos utilizados no corpo do request, seus significados e algumas de suas características:

| Parâmetro | Tipo | Obrigatório? | Padrão | Descrição |
|-----------|------|--------------|--------|-----------|
|```prompt```| string ou array | Opcional |```<\|endoftext\|>```|O(s) prompt(s) para o(s) qual(is) será(ão) gerado(s) as complementações. Note que ```<\|endoftext\|>``` é o separador do documento que o modelo enxerga durante o treinamento. Se um prompt não é especificado, então o modelo irá gerar a partir do começo de um novo documento. |
|```max_tokens```| integer | Opcional | 16 | O número máximo de tokens para gerar a complementação. A contagem de tokens não pode exceder o tamanho do contexto do modelo. A maioria dos modelos tem um comprimento de contexto de 2048 tokens. |
|```temperature```| number | Opcional | 1 | Qual a temperatura a ser utilizada, variando entre 0 e 2. Valores maiores significam que o modelo tomará mais riscos. Tente 0.9 para aplicações mais criativas e 0 (argmax sampling) para modelos com respostas mais bem definidas. Recomenda-se alterar a temperatura ou o top_p, mas não ambos. |
|```top_p```| number | Opcional | 1 | Uma alternativa à temperatura, chamada nucleus sampling, onde o modelo considera os resultados dos tokens com probabilidade top_p de massa. Então 0,1 significa que apenas os tokens dentro de 10% da probabilidade de massa serão considerados. ecomenda-se alterar a temperatura ou o top_p, mas não ambos. Os valores variam entre 0 e 2. |
|```logit_bias```| map | Opcional | null | Modifica as chances de um token aparecer na complementação. Aceita um objeto json que mapeia tokens (especificados por seu token ID do GPT tokenizer) e um valor associado de viés entre -100 e 100. Pode-se utilizar esse tokenizador (GPT-2 e GPT-3) para converter texto para token ID's. Matematicamente, o viés é adicionado aos logits gerados pelo modelo antes da amostragem. Os efeitos exatos variam por modelo, mas valores entre -1 e 1 podem diminuir ou aumentar as chances de seleção; já valores como -100 e 100 podem resultar em banimento ou na seleção exclusiva de certos tokens. Por exemplo, você pode passar {'50256': -100} para previnir que o token ```<\|endoftext\|>``` seja gerado. |
|```user```| string | Opcional | | Um identificador único representando seu usuário, que pode ajudar a identificar e monitorar abuso. |
|```n```| integer | Opcional | 1 | Quantas complementações serão geradas para cada prompt. Nota: esse parâmetro pode aumentar bastante o número de complementações, o que pode acabar rapidamente com a cota de tokens.
|```stream```| boolean | Optional | False | Se o progresso parcial deve ser transmitido de volta. Se definido, os tokens serão enviados como eventos enviados pelo servidor somente de dados assim que estiverem disponíveis, com o fluxo finalizado por uma mensagem data: [DONE]. |
|```logprobs```| integer | Optional | null | Inclue o log de probabilidades dos tokens mais prováveis, assim como dos tokens escolhidos. Por exemplo, se o logprobs é 10, a API retornará uma lista dos 10 tokens mais prováveis. A API sempre retornará o logprob dos tokens da amostra, então haverá logprobs+1 elementos na resposta. Esse parâmetro não pode ser utilizado com o modelo ```gpt-35-turbo```. |
|```suffix```| string | Opcional | null | O sufixo que vem após uma complementação de texto. |
|```echo```| boolean | Opcional | False | Um prefixo adicionado antes do prompt para a complementação. |
|```stop```| string ou array | Opcional | null | Até quatro sequências nas quais a API para de gerar mais tokens. O texto retornado não irá conter as stop sequences. |
|```presence_penalty```| number | Opcional | 0 | Número entre -2.0 e 2.0. Valores positivos penalizam nos tokens baseados onde eles aparecem no texto, aumentando a chance do modelo escrever sobre novos tópicos. |
|```frequency_penalty```| number | Opcional | 0 | Número entre -2.0 e 2.0. Valores positivos penalizam os novos tokens baseados na frequência pela a qual eles aparecem no texto até aquele momento, diminuindo a probabilidade do modelo de repetir as mesmas falas. |
|```best_of```| integer | Opcional | 1 |  Gera as best_of conclusões do lado do servidor e retorna a melhor (aquela com a menor probabilidade de log por token).|

## Temperature

Controla a aleatoriedade. Abaixar a temperatura significa que o modelo irá produzir respostas mais repetitivas e determinísticas. Aumentar a temperatura resultará em respostas mais criativas ou inesperadas. Pode-se ajustar a temperatura ou o Top P, mas não ambos. Os valores de temperatura vão de 0 a 1. Abaixo, iremos testar algumas variações dos valores de temperatura e analisar as respostas da API.

In [3]:
prompt = "Defina o conceito da Física de temperatura:"

cold_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    temperature=0,
    max_tokens=100
)

normal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    temperature=0.5,
    max_tokens=100
)

hot_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    temperature=1,
    max_tokens=100
)

print(f"Temperatura 0: {cold_response['choices'][0]['text']}\n")
print(f"Temperatura 0.5: {normal_response['choices'][0]['text']}\n")
print(f"Temperatura 1: {hot_response['choices'][0]['text']}\n")

Temperatura 0:  A temperatura é uma grandeza física que mede o grau de agitação das moléculas de um corpo. Quanto maior a temperatura, maior a agitação das moléculas e, portanto, maior a energia cinética média do sistema.

O que é a escala Celsius? A escala Celsius é uma escala termométrica que mede a temperatura em graus Celsius (°C). Ela é definida de tal forma

Temperatura 0.5:  A temperatura é uma grandeza física que mede o grau de agitação das partículas que compõem um corpo.

O que é a escala Kelvin? A escala Kelvin é a escala de temperatura termodinâmica, onde o zero absoluto é a menor temperatura possível na natureza.

O que é a escala Celsius? A escala Celsius é uma escala de temperatura que é baseada no ponto de fusão do gelo (0°C

Temperatura 1:  “A ESTATÍSTICA DESCREVE AS PROPRIEDADES DOS MATERIAIS EM FUNÇÃO DA TEMPERATURA”.

11 PROPRIEDADES FÍSICAS DOS MATERIAIS - pressão - volume

O volume e a pressão de um gás estão relacionados pela Teoria Cinética dos Gases. Como se en

Pudemos perceber, que com o aumento do valor da temperatura, a resposta foi menos objetiva. Esses resultados reforçam o conceito abordado acima.

## Max Length (tokens)

Configura o número limite de tokens por requisição do modelo. A API suporta um máximo de 4000 tokens entre os utilizados no prompt (incluindo as mensagens de sistema, exemplos, mensagens de histórico, e query de usuário) e a resposta do modelo. Um token é mais ou menos 4 caracteres para um típico texto. Os valores variam entre 1 e 4000 tokens.

In [4]:
prompt = "Me diga como calcular a área de um círculo"

short_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    max_tokens=10
)

medium_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    max_tokens=100
)

long_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    max_tokens=1000
)

maximum_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    max_tokens=1000
)

print(f"Max Tokens 10: {short_response['choices'][0]['text']}. Número total de tokens:{short_response['usage']['total_tokens']}\n")
print(f"Max Tokens 100: {medium_response['choices'][0]['text']}. Número total de tokens:{medium_response['usage']['total_tokens']}\n")
print(f"Max Tokens 1000: {long_response['choices'][0]['text']}. Número total de tokens:{long_response['usage']['total_tokens']}\n")
print(f"Max Tokens 4000 (máximo): {maximum_response['choices'][0]['text']}. Número total de tokens:{maximum_response['usage']['total_tokens']}\n")

Max Tokens 10:  de raio R.

*A= π R². Número total de tokens:23

Max Tokens 100:  de raio R?

Bom Ana, a área de um círculo é determinada conforme a fórmula abaixo:

A = ∏ R²

Sendo que "A" é a área do círculo e "R" é o raio do círculo

Exemplo: Calcular a área de um círculo cujo o raio é de 4 cm

A = ∏ R²

A = 3,14. Número total de tokens:113

Max Tokens 1000: ");
    chatbot.tfidf_model.add_example('como calcular area de um circulo', 'Calculate the area of a spin');

    test("Calculo de Porcentagem", chatbot.ask("Como calcular o valor de 10% de 100?"));
    test("Calculo de Porcentagem", chatbot.ask("Qual é o valor de 25% de 200?"));
    test("Calculo de Porcentagem", chatbot.ask("Quanto é 50% de R$500,00?"));
    test("Calculo de Porcentagem", chatbot.ask("25% é igual a quantos décimos?"));
    test("Calculo de Porcentagem", chatbot.ask("Qual é o aumento percentual de R$50,00 para R$180,00?"));

    console.log("----> Chatbot Frontend");

    var url = 'http://localhost:8000/api/ch

## Stop Sequences

Faz com que as respostas parem em um ponto desejado, como o fim de uma sentença, ou uma lista. Especifica-se até quatro sequências nas quais o modelo irá parar de gerar tokens na resposta. O texto retornado não contém as sequências de parada.

In [5]:
prompt="""O diálogo a seguir é uma conversação com um assistente de Inteligência Artificial. O assistente é prestativo, criativo, esperto e muito amigável.
\n \nHuman: Olá, quem é você?
\nAI: Olá, eu sou um assistente inteligente. Estou aqui pra ajudá-lo com algo que você precise.
\nHuman: Eu gostaria de cancelar minha inscrição.
\nAI:"""
stop_sequences=["Human:","AI:"]

chat_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    stop=stop_sequences,
    max_tokens=256
)

chat_response

<OpenAIObject text_completion id=cmpl-7gjhlfzw6OlBkN6HM9J32PTpJx8JD at 0x2b2002edee0> JSON: {
  "id": "cmpl-7gjhlfzw6OlBkN6HM9J32PTpJx8JD",
  "object": "text_completion",
  "created": 1690419821,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": " Claro, sem problemas. Voc\u00ea est\u00e1 se referindo para uma inscri\u00e7\u00e3o espec\u00edfica ou a todas elas?\n\n",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 90,
    "total_tokens": 114
  }
}

## Top Probabilities

Similar à temperatura, esse parâmetro controla a aleatoriedade da resposta, mas de uma forma diferente. Abaixar o top P irá retringir a escolha de tokens para os tokens mais prováveis, ou seja, com mais chances de serem o próximo token a completar a sentença. Aumentar o top P permitirá que o modelo escolha tokens com alta e baixa probabilidade. Pode-se ajustar a temperatura ou o top p, mas não ambos. Os valores de top p podem ir de 0 até 1.

In [6]:
prompt = "Defina o conceito de probabilidade:"

cold_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    top_p=0,
    max_tokens=100
)

normal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    top_p=0.5,
    max_tokens=100
)

hot_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    top_p=1,
    max_tokens=100
)

print(f"Top P 0: {cold_response['choices'][0]['text']}\n")
print(f"Top P 0.5: {normal_response['choices'][0]['text']}\n")
print(f"Top P 1: {hot_response['choices'][0]['text']}\n")

Top P 0:  A probabilidade é um ramo da matemática que se dedica ao estudo dos fenômenos aleatórios. Ela é utilizada para se estimar a chance de ocorrência de um determinado evento em uma experiência aleatória.

Qual é a fórmula da probabilidade? A fórmula da probabilidade é dada por: P(A) = n(A) / n(S), onde P(A) é a probabilidade do evento

Top P 0.5:  

A probabilidade é uma medida numérica que varia de 0 a 1, que tem como objetivo medir a chance de um evento ocorrer.

- Quais são os métodos clássico e frequentista para calcular probabilidade?

O método clássico é utilizado para calcular a probabilidade de eventos que possuem um espaço amostral finito e equiprovável. A probabilidade é calculada dividindo-se o número de eventos favoráveis pelo número total de

Top P 1: }{A probabilidade é um conceito tão complexo que pode ser apresentado de diversas formas. Em uma definição mais matemática:
\begin{itemize}
\item[\textbf{Definição}] Dado um espaço amostral $\Omega$, uma probabilidade é

## Frequency Penalty

Reduz a chance de repetir um token proporcionalmente baseado em quão frequente ele aparece no texto até agora. Isso diminui a chance de repetir o exatamente o mesmo texto em uma reposta. Os valores variam de 0 até 2.

In [10]:
prompt = "O que é preciso para desenvolver um chatbot e como eu posso fazer isso?"

low_freq_penal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    frequency_penalty=0,
    max_tokens=200
)

normal_freq_penal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    frequency_penalty=1,
    max_tokens=200
)

high_freq_penal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    frequency_penalty=2,
    max_tokens=200
)

print(f"Frequency Penalty 0: {low_freq_penal_response['choices'][0]['text']}\n")
print(f"Frequency Penalty 1: {normal_freq_penal_response['choices'][0]['text']}\n")
print(f"Frequency Penalty 2: {high_freq_penal_response['choices'][0]['text']}\n")

Frequency Penalty 0:  Esta é uma de nossas perguntas […]

Leia mais…

O WhatsApp é um aplicativo incrível para comunicação pessoal e empresarial, porém, ainda não há segurança de que suas informações […]

Você já deve ter ouvido falar sobre API. API é uma sigla em inglês que significa “Application Programming Interface”. Em […]

Na hora de utilizar aplicativos mobile para negócios, sempre devemos ter em mente: facilidade de uso, layout amigável e boa […]

Com a constante atualização da tecnologia e a facilidade no acesso a itens de consumo, é necessário que as marcas […]

Os usuários de aplicativos móveis estão aumentando a cada dia. Estima-se que haja cerca de 2 bilhões de usuários de […]

Quando se trata de negócios e relacionamento com o cliente, a segurança do aplicativo é crucial. Se você estiver procurando […]

Frequency Penalty 1:  As soluções ainda dependem em grande parte do uso de análises e dados disponíveis para aprimorar suas capacidades.

Acer inovou com projetor 4K holog

## Presence Penalty

Reduz a chance de repetir qualquer token que apareceu no texto. Isso aumenta a probabilidade de introduzir novos tópicos em uma resposta. Os valores variam de 0 até 2.

In [11]:
prompt = "O que é preciso para desenvolver um chatbot e como eu posso fazer isso?"

low_pres_penal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    presence_penalty=0,
    max_tokens=200
)

normal_pres_penal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    presence_penalty=1,
    max_tokens=200
)

high_pres_penal_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=prompt,
    presence_penalty=2,
    max_tokens=200
)

print(f"Presence Penalty 0: {low_pres_penal_response['choices'][0]['text']}\n")
print(f"Presence Penalty 1: {normal_pres_penal_response['choices'][0]['text']}\n")
print(f"Presence Penalty 2: {high_pres_penal_response['choices'][0]['text']}\n")

Presence Penalty 0:  Como ele possuirá a capacidade de dialogar com o usuário, qual tipo de inteligência artificial eu posso utilizar?

A partir do seu conhecimento no desenvolvimento de software, você pode facilmente construir um ChatBot chamado TesteBot, usando Node.JS e um serviço do Watson da IBM com a tecnologia de processamento de linguagem natural chamada de Conversation.

Seguindo os princípios do JavaScript e da arquitetura REST, Node.JS é uma plataforma para desenvolvimento de aplicações web escaláveis e de alta performance, que roda na máquina onde está o servidor, geralmente, com o conjunto de serviços de banco de dados.

A proposta da IBM com o Conversational ajuda a entender mensagens dos usuários e permitir que o chatbot reflita sobre padrões de fala, tendo a habilidade de entender e gerir um diálogo, gerar questionamentos para ação futura

Presence Penalty 1:  Preciso ter conhecimentos em programação?”.

A Bolha de Sabão Comunicação Inteligente é conhecedora dos melhore

## Pre e Pos-response text

O pre-response text é inserido depois da entrada do usuário e antes da resposta do modelo. Isso pode preparar o modelo para uma resposta. Por sua vez, o pos-response text é inserido depois da resposta gerada pelo modelo, levando o usuário a oferecer uma entrada em seguinda, da forma que se faz em uma conversação.